# softmax classifier

In [3]:
import tensorflow as tf

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
# 3가지 class 중 하나만 나타내는 방법으로 "one-hot encoding"방법을 사용한다
# 하나의 class만 1로 표현하고 나머지는 0으로 나타내는 방식
y_data = [[0, 0, 1], # 2
          [0, 0, 1], # 2
          [0, 0, 1], # 2
          [0, 1, 0], # 1
          [0, 1, 0], # 1
          [0, 1, 0], # 1
          [1, 0, 0], # 0
          [1, 0, 0]] # 0

X = tf.placeholder("float",[None,4])
Y = tf.placeholder("float",[None,3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4,nb_classes]), name = "weight")
b = tf.Variable(tf.random_normal([nb_classes]),name = "bias")

# tf.nn.softmax compute softmax activations
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# entrophy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis = 1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
            _, cost_val = sess.run([optimizer, cost], feed_dict={X: x_data, Y: y_data})

            if step % 200 == 0:
                print(step, cost_val)

    print('--------------')
    # Testing & One-hot encoding
    # hypothesis를 run시키면 softmax를 통과하여 각 class에 대한 확률이 출력된다
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    # argmax() : 출력된 확률 중 가장 높은 값의 인덱스를 가져 오고 1을 준다
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')
    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))


0 4.334053
200 0.5836767
400 0.48017296
600 0.40384805
800 0.33453333
1000 0.2637388
1200 0.22255583
1400 0.20263784
1600 0.18587577
1800 0.17158398
2000 0.15926342
--------------
[[6.2753637e-03 9.9371248e-01 1.2098507e-05]] [1]
--------------
[[0.8436803  0.13292445 0.02339527]] [0]
--------------
[[2.0804881e-08 3.8738400e-04 9.9961263e-01]] [2]
--------------
[[6.2753516e-03 9.9371248e-01 1.2098497e-05]
 [8.4368038e-01 1.3292433e-01 2.3395257e-02]
 [2.0804881e-08 3.8738397e-04 9.9961263e-01]] [1 0 2]


# Fancy Softmax Classifier

In [8]:
import tensorflow as tf
import numpy as np

xy = np.loadtxt('C:\python\data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7 # 0~6

X = tf.placeholder(tf.float32,[None,16])
Y = tf.placeholder(tf.int32,[None,1])

# 0~6까지 숫자인 y_data를 one_hot으로 변경한다
Y_one_hot = tf.one_hot(Y, nb_classes)
print(Y_one_hot)
# one_hot으로 변경하면 shape이 3차원이 되서 원하는 shape으로 reshape해줘야 한다
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print(Y_one_hot)

W = tf.Variable(tf.random_normal([16,nb_classes]), name = "weight")
b = tf.Variable(tf.random_normal([nb_classes]), name = "bias")

logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

# tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)                       

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# argmax() : probabilities 중 제일 큰 값을 선택하여 하나의 값(0 ~ 6)으로 출력한다                
prediction = tf.argmax(hypothesis, 1)
# tf.argmax(Y_one_hot, 1) : one_hot을 하나의 값(label)로 나타낸다
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
# 정확하게 판단한 correct_prediction들을 float로 cast하여 모아서 평균울 낸다
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
                                        
        if step % 100 == 0:
            print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%}".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step:     0	Cost: 4.434	Acc: 5.88%
Step:   100	Cost: 0.194	Acc: 94.12%
Step:   200	Cost: 0.091	Acc: 100.00%
Step:   300	Cost: 0.054	Acc: 100.00%
Step:   400	Cost: 0.038	Acc: 100.00%
Step:   500	Cost: 0.029	Acc: 100.00%
Step:   600	Cost: 0.023	Acc: 100.00%
Step:   700	Cost: 0.019	Acc: 100.00%
Step:   800	Cost: 0.017	Acc: 100.00%
Step:   900	Cost: 0.014	Acc: 100.00%
Step:  1000	Cost: 0.013	Acc: 100.00%
Step:  1100	Cost: 0.012	Acc: 100.00%
Step:  1200	Cost: 0.011	Acc: 100.00%
Step:  1300	Cost: 0.010	Acc: 100.00%
Step:  1400	Cost: 0.009	Acc: 100.00%
Step:  1500	Cost: 0.008	Acc: 100.00%
Step:  1600	Cost: 0.008	Acc: 100.00%
Step:  1700	Cost: 0.007	Acc: 100.00%
Step:  1800	Cost: 0.007	Acc: 100.00%
Step:  1